In [ ]:
import pandas as pd

In [ ]:
stats_df = pd.read_csv("../data/tournament_stats_new.csv")

bdo_df = pd.read_csv("../data/BallonDOr_combined.csv")

In [ ]:
year_counts = {}
to_inspect = {'Year': [], 'Rank': []}
for idx, row in bdo_df.iterrows():
    year = row['Year']
    
    year = str(year)[-2:]
    
    year = int(year)

    if 8 <= year <= 23:
        to_inspect['Year'].append(year)
        to_inspect['Rank'].append(row['Rank'])
        if year not in year_counts.keys():
            year_counts[year] = 1

        else:
            year_counts[year] += 1

print(year_counts)
balondor_year_ranks = pd.DataFrame(to_inspect)

In [ ]:
balondor_year_ranks[balondor_year_ranks['Year'] == 8]

In [ ]:
bdo_df['Year'] = bdo_df['Year'].apply(lambda x: int(str(x)[-2:]))
bdo_df = bdo_df.query('Year >= 8 & Year <=23').infer_objects().drop(columns=["Club"])
# ballon_dor['Share'] = np.log(ballon_dor['Share'])

stats_df['Season'] = stats_df['Season'].apply(lambda x: int(str(x)[-2:]))
stats_df = stats_df.query('Season >= 8 & Season <=23').infer_objects()

In [ ]:
# data still there
bdo_df[bdo_df['Year'] == 8.0]

In [ ]:
print(bdo_df.shape)
print(stats_df.shape)

In [ ]:
stats_df.head()

In [ ]:
stats_df['Season'].unique()

In [ ]:
pd.get_dummies(stats_df["Competition"])

In [ ]:
bdo_df.head()

In [ ]:
bdo_df[bdo_df["Player"] == "Cristiano Ronaldo"]

In [ ]:
bdo_df.columns

In [ ]:
stats_df.columns

In [ ]:
desired_columns = ['Year', 'Rank', 'Player', 'Club', 'Nationality', 'Points', 'P1',
                   'P2', 'P3', 'P4', 'P5', 'Votes', 'RankPts', 'Share', 'Percent',
                   'Voted', 'Season', 'Goals', 'Assists', 'Yellow Cards', 'Red Cards',
                   'Minutes Played', 'Goals conceded', 'Clean sheets']

In [ ]:
y8 = stats_df[stats_df['Season'] == 8]
cristiano = y8[y8['Player'] == 'Cristiano Ronaldo']
cristiano

In [ ]:
# aidan merge of bdo_df and stats_df

# join by going through each year, and for each player getting their details from both datasets
# and adding it to a merged row instead

merged = dict()
for col in desired_columns:
    merged[col] = []

for year in range(8, 24):
    stats_by_year = stats_df[stats_df['Season'] == year]
    bdo_by_year = bdo_df[bdo_df['Year'] == year]

    balondor_players = bdo_by_year['Player'].unique()
    
    what_count = 0
    for player in balondor_players:
        if player not in stats_by_year['Player'].values:
            what_count += 1
            print(player)
    print(what_count)

In [ ]:
df = pd.merge(bdo_df, stats_df, left_on=["Player", "Year"], right_on=["Player", "Season"]).drop(columns=["Second yellow cards"])
df.info()

In [ ]:
merged_y8 = df[df['Year'] == 8.0]
merged_y8_rank4 = merged_y8[merged_y8['Rank'] == 4]
merged_y8_rank4

In [ ]:
df["Share"]

In [ ]:
df = df.replace('-', 0)
for col in df.columns:
    try:
        df[col] = df[col].astype('float')
    except:
        continue
# df["Minutes played"] = df['Minutes played'].str.removesuffix("'").astype("float")
df["Minutes played"] = df['Minutes played'].astype("float")

In [ ]:
df["Player"].value_counts().head(20)

In [ ]:
for competition in df["Competition"].unique():
    df[f"{competition.replace(' ', '_')}_appearances"] = df[df["Competition"]== competition]["Appearances"].fillna(0)
    df[f"{competition.replace(' ', '_')}_appearances"].fillna(0, inplace=True)

In [ ]:
df.info()

In [ ]:
base_cols = ["Year", "Rank", "Player", "Club", "Nationality"] 
df[base_cols]

In [ ]:
df_max = df.iloc[:, :18].drop(columns=["Competition"])
df_sum = pd.concat([df[base_cols],df.iloc[:, 19:]], axis=1)

In [ ]:
df["Share"]

In [ ]:
df_sum = df_sum.groupby(base_cols).sum()
df_max = df_max.groupby(base_cols).max()

df_final = df_max.merge(df_sum, left_index=True, right_index=True)

In [ ]:
# df_final = df_final.rename(columns={
    # "is_national": "national_comps",
    # "is_international": "international_comps",
    # "is_domestic": "domestic_comps"
# })

In [ ]:
df_final

In [ ]:
df_final.to_csv('../data/player_stats_new.csv')

### Comparing old with new

In [ ]:
df_final = pd.read_csv('../data/player_stats.csv')
df_final[df_final['Year'] == 8.0]

In [ ]:
df_final_new = pd.read_csv('../data/player_stats_new.csv')
df_final_new[df_final_new['Year'] == 9.0]